In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import repos_tools
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


In [2]:
hgn_fpath = '/Users/jonesa7/CTNS/resources/hgnc/hgnc_complete_set.txt'
usecols = ['entrez_id', 'symbol']
hgn = pd.read_csv(hgn_fpath, sep='\t', usecols=usecols, index_col='symbol', dtype={'entrez_id': str})
hgns = hgn.squeeze()

In [3]:
def symbol2entrezid_line(line, hgns):
    lin = line.rstrip().split('\t')
    lout = lin[:2] + hgns.loc[lin[2:]].to_list()
    sout = '\t'.join(lout) + '\n'
    return(sout)

In [4]:
%connect_info

{
  "shell_port": 53932,
  "iopub_port": 53933,
  "stdin_port": 53934,
  "control_port": 53936,
  "hb_port": 53935,
  "ip": "127.0.0.1",
  "key": "ecd474d9-d3d24261d724fdba1ecde842",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-b38bb719-4cb0-4c21-a2e0-ed0b25b1f1cf.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
